In [ ]:
import cv2
import time
import csv
import serial
import math
import random
import struct
from threading import Thread

class PsEye(Thread):
    def __init__(self):
        super().__init__()
        self._run = True
        self._video = cv2.VideoCapture(0)
        self._last_frame = None

    def get_act_frame(self):
        return self._last_frame

    def run(self):
        while self._run:
            check, frame = self._video.read()
            if check:
                self._last_frame = frame

    def save_last_frame(self, img_file_path, img_name):
        print('Saving img {}'.format(img_name))
        cv2.imwrite(str(img_file_path + img_name), self._last_frame)

    def end(self):
        self._run = False
        self.join()
        self._video.release()

    def frame_valid(self):
        return self._last_frame is not None

Camera = PsEye()
Camera.start()

# Waiting until camera ready
while not Camera.frame_valid():
    time.sleep(0.05)

print('Camera Running...')

# list available ports:
# $ l

ser = serial.Serial('/dev/ttyACM3',115200)
csv_file_path = "./image_labels.csv"
img_file_path = "./pictures/"


def move(random_rad):
    ser.write(str(random_rad).encode('utf-8'))

def get_label_value():
    ser.flushInput()
    value = ser.read_until()
    time.sleep(0.005)
    print(value)
    return float(value)

def take_picture(img_file_path, img_name):
    Camera.save_last_frame(img_file_path, img_name)
    
def log_csv(file, img_name, label):
    with open(file, 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow([img_name, label])
    

for a in range(20):  
    rand_increment = random.randint(100,200) 
    rand_rad = rand_increment/4096 * (2*math.pi)
    move(rand_rad)
    time.sleep(1.2)
    label = get_label_value()
    label =label %(2*math.pi)
    img_name = "{}_{}.png".format(label, time.time())
    take_picture(img_file_path, img_name)
    log_csv(csv_file_path, img_name, label)

Camera.end()
ser.close()